In [ ]:
import requests
import json
from datetime import date, timedelta

In [1]:
# Mount ADLS Gen2
# Required each time the cluster is restarted which should be only on the first notebook as they run in order

tiers = ["bronze", "silver", "gold"]
adls_paths = {tier: f"abfss://{tier}@your_storage_account_name.dfs.core.windows.net/" for tier in tiers}

# Accessing paths
bronze_adls = adls_paths["bronze"]
silver_adls = adls_paths["silver"]
gold_adls = adls_paths["gold"]

StatementMeta(synapsesApache, 6, 2, Finished, Available, Finished)

In [2]:
# files = mssparkutils.fs.ls(bronze_adls)
# for file in files:
#     print(file.name, file.isDir, file.isFile, file.path, file.size, file.modifyTime)

StatementMeta(synapsesApache, 6, 3, Finished, Available, Finished)

2025-04-01_earthquake_data.json True False abfss://bronze@apistoragewem.dfs.core.windows.net/2025-04-01_earthquake_data.json 0 1743619791831


In [4]:
start_date = date.today() - timedelta(1)
end_date = date.today()
start_date, end_date

StatementMeta(synapsesApache, 6, 5, Finished, Available, Finished)

In [6]:
# Construct the API URL with start and end dates provided by Data Factory
url = f"https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime={start_date}&endtime={end_date}"

try:
    # Make the GET request to fetch data
    response = requests.get(url)

    # Check if the request was successful
    response.raise_for_status()  # Raise HTTPError for bad responses (4xx or 5xx)
    data = response.json().get('features', [])

    if not data:
        print("No data returned for the specified date range.")
    else:
        # Specify the ADLS path
        file_path = f"{bronze_adls}/{start_date}_earthquake_data.json"

        # Convert data to JSON string
        json_data = json.dumps(data, indent=4)

        # Write the JSON data to ADLS
        # Create an RDD with the JSON string and parallelize it
        rdd = spark.sparkContext.parallelize([json_data])

        # Convert RDD to DataFrame and write to ADLS
        df = spark.read.json(rdd)
        df.limit(3) # To speed up processing for tutorial
        df.write.mode("overwrite").json(file_path)

        print(f"Data successfully saved to {file_path}")
except requests.exceptions.RequestException as e:
    print(f"Error fetching data from API: {e}")

StatementMeta(synapsesApache, 6, 7, Finished, Available, Finished)

Data successfully saved to abfss://bronze@apistoragewem.dfs.core.windows.net//2025-04-03_earthquake_data.json


In [7]:
data[0]

StatementMeta(synapsesApache, 6, 8, Finished, Available, Finished)

{'type': 'Feature',
 'properties': {'mag': 4.4,
  'place': '49 km NE of Al Jubayl, Saudi Arabia',
  'time': 1743723576440,
  'updated': 1743771830285,
  'tz': None,
  'url': 'https://earthquake.usgs.gov/earthquakes/eventpage/us6000q3ua',
  'detail': 'https://earthquake.usgs.gov/fdsnws/event/1/query?eventid=us6000q3ua&format=geojson',
  'felt': 6,
  'cdi': 3.8,
  'mmi': None,
  'alert': None,
  'status': 'reviewed',
  'tsunami': 0,
  'sig': 300,
  'net': 'us',
  'code': '6000q3ua',
  'ids': ',us6000q3ua,',
  'sources': ',us,',
  'types': ',dyfi,origin,phase-data,',
  'nst': 47,
  'dmin': 5.546,
  'rms': 0.69,
  'gap': 72,
  'magType': 'mb',
  'type': 'earthquake',
  'title': 'M 4.4 - 49 km NE of Al Jubayl, Saudi Arabia'},
 'geometry': {'type': 'Point', 'coordinates': [49.9597, 27.3505, 10]},
 'id': 'us6000q3ua'}

In [8]:
# Define your variables
output_data = {
    "start_date": start_date.isoformat(),
    "bronze_adls": bronze_adls,
    "silver_adls": silver_adls,
    "gold_adls": gold_adls
}

# Serialize the dictionary to a JSON string
bronze_output = json.dumps(output_data)
# print(bronze_output)

# Use mssparkutils.notebook.exit() to pass the JSON output to the pipeline
mssparkutils.notebook.exit(bronze_output)

StatementMeta(synapsesApache, 6, 9, Finished, Available, Finished)

{"start_date": "2025-04-03", "bronze_adls": "abfss://bronze@apistoragewem.dfs.core.windows.net/", "silver_adls": "abfss://silver@apistoragewem.dfs.core.windows.net/", "gold_adls": "abfss://gold@apistoragewem.dfs.core.windows.net/"}
ExitValue: {"start_date": "2025-04-03", "bronze_adls": "abfss://bronze@apistoragewem.dfs.core.windows.net/", "silver_adls": "abfss://silver@apistoragewem.dfs.core.windows.net/", "gold_adls": "abfss://gold@apistoragewem.dfs.core.windows.net/"}